<h1 align="center"  style="font-family:Fontin;color:#000066;font-size:24pt">
Training neural network with single hidden layer of rectified linear units with 2048 nodes on notMNIST</h1> 


In [ ]:
 
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

# load the datasets and lables from previously saved pickle.

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print 'Training set: ', train_dataset.shape, train_labels.shape
    print 'Validation set: ', valid_dataset.shape, valid_labels.shape
    print 'Test set: ', test_dataset.shape, test_labels.shape

# Reformat the image files in to flat vector
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'After reformatting the data'
print 'Training set: ', train_dataset.shape, train_labels.shape
print 'Validation set: ', valid_dataset.shape, valid_labels.shape
print 'Test set: ', test_dataset.shape, test_labels.shape    



batch_size = 1024

graph = tf.Graph()
with graph.as_default():

    #tf Placeholders for minibatch train data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    #tf constants for valid and test data
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    non=2048 # number of nodes in relu layer
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, non]))
    biases = tf.Variable(tf.zeros([non]))    

    # hidden relu layer
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)

    weights_2 = tf.Variable(tf.truncated_normal([non, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    logits= tf.matmul(hidden, weights_2) + biases_2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) 
                                                          + biases), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) 
                                                         + biases), weights_2) + biases_2)


# define accuracy function     
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])    
    
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        # generate minibatch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 1000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Training set:  (400000, 28, 28) (400000,)  
Validation set:  (100000, 28, 28) (100000,)   
Test set:  (18000, 28, 28) (18000,)  

After reformatting the data  
Training set:  (400000, 784) (400000, 10)  
Validation set:  (100000, 784) (100000, 10)  
Test set:  (18000, 784) (18000, 10)  

Minibatch loss at step 0: 460.899506  
Minibatch accuracy: 10.3%  
Validation accuracy: 39.2%  
Minibatch loss at step 1000: 8.656817  
Minibatch accuracy: 86.9%  
Validation accuracy: 85.6%  
Minibatch loss at step 2000: 8.440260  
Minibatch accuracy: 87.3%  
Validation accuracy: 86.3%  
Minibatch loss at step 3000: 10.808512  
Minibatch accuracy: 85.5%  
Validation accuracy: 82.6%  
Minibatch loss at step 4000: 2.671515  
Minibatch accuracy: 87.8%  
Validation accuracy: 87.0%  
Minibatch loss at step 5000: 2.190775  
Minibatch accuracy: 89.6%  
Validation accuracy: 87.4%  
Minibatch loss at step 6000: 2.016960  
Minibatch accuracy: 89.8%  
Validation accuracy: 86.6%  
Minibatch loss at step 7000: 1.231713  
Minibatch accuracy: 90.6%  
Validation accuracy: 87.4%  
Minibatch loss at step 8000: 1.144581  
Minibatch accuracy: 90.6%  
Validation accuracy: 87.7%  
Minibatch loss at step 9000: 1.123389  
Minibatch accuracy: 92.6%  
Validation accuracy: 87.4%  
Minibatch loss at step 10000: 0.785717  
Minibatch accuracy: 91.0%  
Validation accuracy: 87.9%  
Test accuracy: 93.6%  
